# Shakespeare Transformer - Interactive Capstone

Complete end-to-end training of a character-level transformer on Shakespeare text.

**What You'll Learn:**
- Data preparation and tokenization
- Model configuration and training
- Text generation with different sampling strategies
- Model evaluation and analysis

**Time**: ~10-15 minutes (on GPU/Colab with reduced steps)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/tiny-transformer-course/blob/main/docs/modules/09_capstone/notebook.ipynb)

## 1. Setup & Installation

Install dependencies and setup environment.

In [ ]:
# Verify PyTorch
import torch
import sys
from pathlib import Path

print(f'Python: {sys.version}')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'Device: {\'cuda\' if torch.cuda.is_available() else \'cpu\'}')

# Add project to path (if running locally)
if Path('../../../tiny_transformer').exists():
    sys.path.insert(0, str(Path.cwd().parent.parent.parent))
    print('✓ Running locally')
else:
    print('Note: Adjust paths for your environment')

## 2. Data Preparation

Download and analyze the Shakespeare dataset.

In [ ]:
# Load Shakespeare dataset
data_path = '../../../data/tiny_shakespeare.txt'

with open(data_path, 'r') as f:
    text = f.read()

print(f'Total characters: {len(text):,}')
print(f'Total lines: {text.count(chr(10)):,}')
print(f'Vocabulary size: {len(set(text))}')
print(f'\nFirst 500 characters:')
print(text[:500])

In [ ]:
# Character distribution analysis
from collections import Counter
import matplotlib.pyplot as plt

char_counts = Counter(text)
top_chars = char_counts.most_common(20)

chars = [repr(c) if c in ['\n', '\t', ' '] else c for c, _ in top_chars]
counts = [count for _, count in top_chars]

plt.figure(figsize=(12, 6))
plt.bar(chars, counts)
plt.xlabel('Character')
plt.ylabel('Frequency')
plt.title('Top 20 Characters in Shakespeare Dataset')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Tokenization

Create character-level tokenizer.

In [ ]:
from tiny_transformer.training import CharTokenizer

# Create and fit tokenizer
tokenizer = CharTokenizer()
tokenizer.fit(text)

print(f'Vocabulary size: {tokenizer.vocab_size}')
print(f'\nFirst 10 vocab items:')
for char, idx in list(tokenizer.vocab.items())[:10]:
    print(f'  {repr(char)}: {idx}')

In [ ]:
# Test encoding/decoding
test_text = 'ROMEO: What say you?'
tokens = tokenizer.encode(test_text)
decoded = tokenizer.decode(tokens)

print(f'Original: {test_text}')
print(f'Tokens: {tokens}')
print(f'Decoded: {decoded}')
print(f'Match: {test_text == decoded}')

## 4. Model Configuration

Setup model and training configuration.

In [ ]:
from tiny_transformer.model import TinyTransformerLM

# Model configuration (smaller for faster training)
config = {
    'vocab_size': tokenizer.vocab_size,
    'd_model': 256,
    'n_heads': 4,
    'n_layers': 4,
    'd_ff': 1024,
    'max_seq_len': 128,
    'dropout': 0.1,
    'tie_weights': True,
}

# Create model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TinyTransformerLM(**config).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(f'Model parameters: {num_params:,}')
print(f'Device: {device}')

## 5. Training Setup

Prepare datasets and training components.

In [ ]:
from tiny_transformer.training import TextDataset
from torch.utils.data import DataLoader

# Tokenize full text
tokens = tokenizer.encode(text)

# Split train/val (90/10)
val_size = len(tokens) // 10
train_tokens = tokens[:-val_size]
val_tokens = tokens[-val_size:]

print(f'Train tokens: {len(train_tokens):,}')
print(f'Val tokens: {len(val_tokens):,}')

# Create datasets
train_dataset = TextDataset(train_tokens, seq_len=128, stride=64)
val_dataset = TextDataset(val_tokens, seq_len=128, stride=128)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')

In [ ]:
from tiny_transformer.training import Trainer, TrainerConfig, WarmupCosineScheduler

# Training configuration
trainer_config = TrainerConfig(
    learning_rate=1e-3,
    weight_decay=0.01,
    grad_clip=1.0,
    warmup_steps=200,
    max_steps=2000,
    eval_interval=200,
    log_interval=50,
    device=device,
)

# Create trainer
trainer = Trainer(model, train_loader, val_loader, trainer_config)

# Setup scheduler
scheduler = WarmupCosineScheduler(
    optimizer=trainer.optimizer,
    warmup_steps=trainer_config.warmup_steps,
    total_steps=trainer_config.max_steps,
    peak_lr=trainer_config.learning_rate,
    min_lr=1e-5,
)
trainer.scheduler = scheduler

print('Training setup complete!')

## 6. Training

Train the model (this will take ~10-15 minutes on GPU).

In [ ]:
# Train with progress tracking
import time

print('='*70)
print('Starting Training')
print('='*70)

start_time = time.time()

try:
    metrics = trainer.train()
    
    elapsed = time.time() - start_time
    print(f'\nTraining complete in {elapsed/60:.1f} minutes')
    print(f'Final train loss: {metrics.get("train_loss", 0):.3f}')
    print(f'Final val loss: {metrics.get("val_loss", 0):.3f}')
    
except KeyboardInterrupt:
    print('\nTraining interrupted')

## 7. Text Generation

Generate Shakespeare-style text with the trained model.

In [ ]:
from tiny_transformer.sampling import TextGenerator, GeneratorConfig

def generate(prompt, temperature=0.8, max_tokens=150):
    """Generate text from prompt."""
    gen_config = GeneratorConfig(
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.95,
        do_sample=True,
    )
    
    generator = TextGenerator(model, gen_config, device=device)
    
    model.eval()
    prompt_tokens = torch.tensor([tokenizer.encode(prompt)]).to(device)
    
    with torch.no_grad():
        output_tokens = generator.generate(prompt_tokens)
    
    return tokenizer.decode(output_tokens[0].tolist())

# Generate sample
prompt = 'ROMEO:'
generated = generate(prompt)

print('='*70)
print(f'Prompt: {prompt}')
print('-'*70)
print(generated)
print('='*70)

## 8. Compare Sampling Strategies

Experiment with different sampling parameters.

In [ ]:
# Compare different temperatures
prompt = 'HAMLET:'
temperatures = [0.5, 0.8, 1.2]

for temp in temperatures:
    print(f'\nTemperature: {temp}')
    print('-'*70)
    generated = generate(prompt, temperature=temp, max_tokens=100)
    print(generated)
    print('-'*70)

## 9. Interactive Generation

Try your own prompts!

In [ ]:
# Interactive cell - run multiple times with different prompts
your_prompt = 'JULIET:'  # Change this!
temperature = 0.8  # Adjust creativity
max_tokens = 200  # Length

result = generate(your_prompt, temperature, max_tokens)
print(result)

## 10. Save Model

Save the trained model for later use.

In [ ]:
# Save checkpoint
checkpoint = {
    'model_state_dict': model.state_dict(),
    'config': config,
    'tokenizer_vocab': tokenizer.vocab,
}

torch.save(checkpoint, 'shakespeare_model.pt')
print('✓ Model saved to shakespeare_model.pt')

## Congratulations!

You've successfully trained a character-level transformer on Shakespeare!

**What you learned:**
- Data preparation and tokenization
- Model architecture and configuration
- Training loop with validation
- Text generation with sampling strategies

**Next steps:**
1. Train longer (20k steps) for better quality
2. Try different model sizes (d_model=384, n_layers=6)
3. Fine-tune on your own text data
4. Experiment with advanced sampling (beam search, nucleus sampling)

See `docs/modules/09_capstone/walkthrough.md` for the complete guide!